In [43]:
import pandas as pd
import numpy as np
from core import filter_clientes_importantes, filter_productos_importantes

# 1. Crudo Desglosado

## Cargamos originales

In [44]:
# Reading the CSV file
original_data = pd.read_csv("./datasets/originales_clase_3/tb_sellout_02.txt.gz", sep="\t")
original_categories = pd.read_csv("./datasets/originales_clase_3/tb_productos_02.txt", sep="\t")

In [45]:
original_data

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,201701,10234,20524,0,2,0.05300,0.05300
1,201701,10032,20524,0,1,0.13628,0.13628
2,201701,10217,20524,0,1,0.03028,0.03028
3,201701,10125,20524,0,1,0.02271,0.02271
4,201701,10012,20524,0,11,1.54452,1.54452
...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230
2945814,201912,10092,20853,0,1,0.00669,0.00669
2945815,201912,10006,20853,0,7,0.02898,0.02898
2945816,201912,10018,20853,0,4,0.01561,0.01561


In [46]:
original_categories

,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288
...,...,...,...,...,...,...
1257,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,12,20240
1258,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,6,20285
1259,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,2,20259
1260,FOODS,SOPAS Y CALDOS,Caldo Cubo,MAGGI2,12,20142


In [47]:
unknown_category_product_ids = [20808, 20848, 21066, 21098, 21125, 21165, 21178, 21199, 21217,
       21223, 21225, 21230, 21238, 21240, 21241, 21249, 21253, 21268,
       21272, 21273, 21274, 21275, 21277, 21285, 21286, 21295, 21297,
       21281, 21278, 21284, 21279, 21283, 21289, 21290, 21288, 21291,
       21292, 21293, 21296, 21298, 21299, 20918, 21228, 21270, 21169]
original_categories[original_categories["product_id"].isin(unknown_category_product_ids)]

,cat1,cat2,cat3,brand,sku_size,product_id


In [48]:
original_categories["product_id"].nunique()

1251

In [49]:
# drop duplicate product_id from the original_categories dataframe, keep first row
original_categories.drop_duplicates(subset="product_id", keep="first", inplace=True)
original_categories["product_id"].nunique()

1251

In [50]:
original_data["periodo"].value_counts().sort_index()

periodo
201701     78830
201702     82270
201703     90603
201704     82837
201705     96042
201706     96169
201707     74514
201708     92535
201709     91287
201710     92731
201711     92348
201712     79841
201801     81199
201802     84022
201803    101066
201804     81959
201805     85520
201806     89551
201807     76695
201808     88104
201809     74621
201810     80148
201811     72280
201812     70166
201901     67628
201902     82519
201903     89520
201904     77818
201905     73404
201906     81994
201907     81676
201908     66735
201909     81197
201910     76624
201911     73443
201912     57922
Name: count, dtype: int64

In [51]:
original_data["product_id"].nunique(), original_data["customer_id"].nunique(), original_data["periodo"].nunique() 

(1233, 597, 36)

In [52]:
df = original_data

In [53]:
df[df["plan_precios_cuidados"] == 1]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
15380,201701,10347,20405,1,1,0.03276,0.03276
15381,201701,10174,20405,1,5,0.13104,0.13104
15382,201701,10413,20405,1,1,0.02457,0.02457
15383,201701,10042,20405,1,1,0.04914,0.04914
15384,201701,10050,20405,1,1,0.08190,0.08190
...,...,...,...,...,...,...,...
2944586,201912,10173,20323,1,1,0.03320,0.03320
2944587,201912,10313,20323,1,1,0.01660,0.01660
2944588,201912,10064,20323,1,2,0.07469,0.07469
2944589,201912,10015,20323,1,4,2.32378,2.32378


## Limpiamos types

In [54]:
df['periodo'] = df['periodo'].astype(int)
df['product_id'] = df['product_id'].astype(int)
df['customer_id'] = df['customer_id'].astype(int)
df['plan_precios_cuidados'] = df['plan_precios_cuidados'].astype(int)
df['cust_request_qty'] = df['cust_request_qty'].astype(float)
df['cust_request_tn'] = df['cust_request_tn'].astype(float)
df['tn'] = df['tn'].astype(float)

In [55]:
print(df["cust_request_qty"].sum(), df["cust_request_tn"].sum(), df["tn"].sum())


6329834.0 1353074.0208099994 1324988.5884099994


In [56]:
# print unique number of customer_id, product_id, and period
print(df["product_id"].nunique(), df["customer_id"].nunique(), df["periodo"].nunique())

1233 597 36


## Agrupamos para desagregar

In [57]:
df_desagregado = df.groupby(['periodo', 'customer_id', 'product_id'])[['cust_request_qty', 'cust_request_tn', 'tn']].sum()

In [58]:
print("ORIGINAL|||", df["cust_request_qty"].sum(), df["cust_request_tn"].sum(), df["tn"].sum())
print("DESAGREGADO", df_desagregado["cust_request_qty"].sum(), df_desagregado["cust_request_tn"].sum(), df_desagregado["tn"].sum())

ORIGINAL||| 6329834.0 1353074.0208099994 1324988.5884099994
DESAGREGADO 6329834.0 1353074.0208100001 1324988.5884099994


In [59]:
df_desagregado.reset_index(inplace=True)

In [60]:
product_ids_maestro = original_categories["product_id"].unique().tolist()
product_ids_sellout = df_desagregado["product_id"].unique().tolist()
faltantes_en_sellout = list(set(product_ids_maestro) - set(product_ids_sellout))
faltantes_en_maestro = list(set(product_ids_sellout) - set(product_ids_maestro))
len(faltantes_en_sellout), len(faltantes_en_maestro)

(63, 45)

In [61]:
df_desagregado.sort_values(by=["periodo", "customer_id", "product_id"], inplace=True)
df_desagregado

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn
0,201701,10001,20001,11.0,99.43861,99.43861
1,201701,10001,20002,17.0,90.13504,87.64856
2,201701,10001,20003,25.0,100.21284,100.21284
3,201701,10001,20004,13.0,21.73954,21.73954
4,201701,10001,20006,18.0,31.36770,29.17196
...,...,...,...,...,...,...
2945813,201912,10606,20303,1.0,0.01298,0.01298
2945814,201912,10606,20563,1.0,0.00442,0.00442
2945815,201912,10606,20962,1.0,0.00655,0.00655
2945816,201912,10606,20975,1.0,0.00655,0.00655


In [62]:
print("ORIGINAL|||", df["cust_request_qty"].sum(), df["cust_request_tn"].sum(), df["tn"].sum())
print("DESAGREGADO", df_desagregado["cust_request_qty"].sum(), df_desagregado["cust_request_tn"].sum(), df_desagregado["tn"].sum())

ORIGINAL||| 6329834.0 1353074.0208099994 1324988.5884099994
DESAGREGADO 6329834.0 1353074.0208100001 1324988.5884099994


## Agregamos los 0s donde faltan y eliminamos productos discontinuados

In [63]:
# Inicializa un DataFrame vacío para almacenar los resultados
df_final = pd.DataFrame()

# Empezamos capturando únicamente los productos que existen en el primer periodo
existing_products = df_desagregado[df_desagregado["periodo"] == 201701]["product_id"].unique().tolist()

# Y capturamos todos los clientes (que no se crean, sino que siempre existen todos)
total_clientes = df_desagregado["customer_id"].unique().tolist()

missing_products_dict = {}

for periodo in df_desagregado["periodo"].unique().tolist():
    print(periodo)
    print("existing_products before", len(existing_products))
    
    # 1) get the products that exist in the current period
    this_period_products = df_desagregado[df_desagregado["periodo"] == periodo]["product_id"].unique().tolist()
    print("this_period_products", len(this_period_products))
    for product in this_period_products:
        if product in missing_products_dict:
            del missing_products_dict[product]

    # 2) get the products that are present in existing_products but not in the current period
    missing_products = list(set(existing_products) - set(this_period_products))
    print("missing_products", len(missing_products))
    for product in missing_products:
        if product in missing_products_dict:
            missing_products_dict[product] += 1
        else:
            missing_products_dict[product] = 1

    # 3) update the existing products list
    existing_products = list(set([*existing_products, *this_period_products]))
    print("existing_products after", len(existing_products))

    for product in missing_products_dict:
        if missing_products_dict[product] >= 3:
            # remove the product from the existing products list
            existing_products.remove(product) if product in existing_products else None

    # Crea todas las combinaciones posibles de clientes y productos para el periodo
    combinaciones_periodo = pd.MultiIndex.from_product([[periodo], total_clientes, existing_products], names=["periodo", "customer_id", "product_id"]).to_frame(index=False)

    # Concatena con el DataFrame final
    df_final = pd.concat([df_final, combinaciones_periodo], ignore_index=True)

# Realiza un merge con tu DataFrame original
df_merged = pd.merge(df_final, df_desagregado, on=["periodo", "customer_id", "product_id"], how='left')

# Rellena los NaN con 0 para las columnas necesarias
df_merged[['cust_request_qty', 'cust_request_tn', 'tn']] = df_merged[['cust_request_qty', 'cust_request_tn', 'tn']].fillna(0)

201701
existing_products before 785
this_period_products 785
missing_products 0
existing_products after 785
201702
existing_products before 785
this_period_products 781
missing_products 11
existing_products after 792
201703
existing_products before 792
this_period_products 786
missing_products 16
existing_products after 802
201704
existing_products before 802
this_period_products 784
missing_products 24
existing_products after 808
201705
existing_products before 800
this_period_products 806
missing_products 10
existing_products after 816
201706
existing_products before 812
this_period_products 823
missing_products 8
existing_products after 831
201707
existing_products before 826
this_period_products 826
missing_products 23
existing_products after 849
201708
existing_products before 849
this_period_products 847
missing_products 17
existing_products after 864
201709
existing_products before 862
this_period_products 829
missing_products 41
existing_products after 870
201710
existing_produ

In [64]:
df_merged[df_merged["product_id"] == 20989]

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn
4410383,201710,10001,20989,3.0,0.26645,0.26645
4411267,201710,10002,20989,0.0,0.00000,0.00000
4412151,201710,10003,20989,0.0,0.00000,0.00000
4413035,201710,10004,20989,0.0,0.00000,0.00000
4413919,201710,10005,20989,0.0,0.00000,0.00000
...,...,...,...,...,...,...
19104962,201912,10577,20989,0.0,0.00000,0.00000
19105921,201912,10591,20989,0.0,0.00000,0.00000
19106880,201912,10618,20989,0.0,0.00000,0.00000
19107839,201912,10572,20989,0.0,0.00000,0.00000


In [65]:
missing_products_dict

{21285: 3,
 21286: 3,
 21287: 3,
 21295: 3,
 21297: 3,
 21275: 3,
 21282: 3,
 21230: 3,
 21199: 3,
 21272: 3,
 21277: 3,
 21253: 3,
 21240: 3,
 21268: 3,
 21278: 3,
 21284: 3,
 21158: 3,
 21160: 3,
 21289: 3,
 21165: 3,
 21193: 3,
 21232: 3,
 21241: 3,
 21225: 3,
 21172: 3,
 21274: 3,
 21279: 3,
 21281: 3,
 21283: 3,
 21288: 3,
 21290: 3,
 21291: 3,
 21292: 3,
 21293: 3,
 21294: 3,
 21296: 3,
 21298: 3,
 21299: 3,
 21066: 3,
 21210: 3,
 21223: 3,
 21096: 3,
 21098: 3,
 21100: 3,
 20591: 3,
 21249: 3,
 21009: 3,
 21143: 3,
 21148: 3,
 21149: 3,
 21059: 3,
 21208: 3,
 21095: 3,
 20455: 3,
 21260: 3,
 20498: 3,
 20909: 3,
 20829: 3,
 21215: 3,
 20833: 3,
 21238: 3,
 20861: 3,
 21273: 3,
 20938: 3,
 21213: 3,
 20581: 3,
 21094: 3,
 20717: 3,
 21061: 3,
 21078: 3,
 21081: 3,
 20955: 3,
 21106: 3,
 20613: 3,
 20619: 3,
 20890: 3,
 20769: 3,
 20897: 3,
 21162: 3,
 20550: 3,
 20813: 3,
 20816: 3,
 21091: 3,
 20634: 3,
 20643: 3,
 21189: 3,
 21082: 3,
 21085: 3,
 20707: 3,
 20738: 3,
 20616: 3,

In [66]:
df_merged

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn
0,201701,10001,20480,6.0,0.85598,0.85598
1,201701,10001,20482,1.0,0.75874,0.75874
2,201701,10001,20483,0.0,0.00000,0.00000
3,201701,10001,20484,6.0,0.89664,0.89664
4,201701,10001,20485,2.0,0.73612,0.73612
...,...,...,...,...,...,...
19109368,201912,10582,20474,0.0,0.00000,0.00000
19109369,201912,10582,20476,0.0,0.00000,0.00000
19109370,201912,10582,20477,0.0,0.00000,0.00000
19109371,201912,10582,20478,0.0,0.00000,0.00000


In [67]:
print("ORIGINAL", df["cust_request_qty"].sum(), df["cust_request_tn"].sum(), df["tn"].sum())
print("MERGED||", df_merged["cust_request_qty"].sum(), df_merged["cust_request_tn"].sum(), df_merged["tn"].sum())

ORIGINAL 6329834.0 1353074.0208099994 1324988.5884099994
MERGED|| 6329834.0 1353074.0208099997 1324988.5884100022


In [68]:
# get number of duplicated rows by <period, customer_id, product_id>
df_duplicated = df_merged[df_merged.duplicated(subset=["periodo", "customer_id", "product_id"], keep=False)]
df_duplicated

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn


In [69]:
df_merged.to_csv("./datasets/tb_sellout_02_desglosado_crudo.csv", index=False)

# 2. Add categories info 

In [70]:
df_desagregado_final = df_merged.copy()

In [71]:
df_desagregado_final = pd.merge(
    df_desagregado_final,
    original_categories[["product_id", "cat1", "cat2", "cat3", "brand", "sku_size"]],
    on="product_id",
    how="left",
    validate="many_to_many",
)
df_desagregado_final.rename(columns={"cat1": "product_category"}, inplace=True)
df_desagregado_final["product_category"].fillna("unknown", inplace=True)
df_desagregado_final["cat2"].fillna("unknown", inplace=True)
df_desagregado_final["cat3"].fillna("unknown", inplace=True)
df_desagregado_final["brand"].fillna("unknown", inplace=True)
df_desagregado_final["sku_size"].fillna(0, inplace=True)

In [72]:
print("ORIGINAL|||", df["cust_request_qty"].sum(), df["cust_request_tn"].sum(), df["tn"].sum())
print("DESAGREGADO", df_desagregado_final["cust_request_qty"].sum(), df_desagregado_final["cust_request_tn"].sum(), df_desagregado_final["tn"].sum())

ORIGINAL||| 6329834.0 1353074.0208099994 1324988.5884099994
DESAGREGADO 6329834.0 1353074.0208099997 1324988.5884100022


In [73]:
df_precios_cuidados = df[df["plan_precios_cuidados"] == 1]
# find rows with duplicated combinations of 'periodo' and 'product_id' and remove them
df_precios_cuidados = df_precios_cuidados.drop_duplicates(subset=['periodo', 'product_id'], keep='first')
df_precios_cuidados["product_id"].nunique(), df_precios_cuidados.shape

(42, (308, 7))

In [74]:
df_precios_cuidados = df_precios_cuidados[['periodo', 'product_id', 'plan_precios_cuidados']]
df_precios_cuidados

,periodo,product_id,plan_precios_cuidados
15380,201701,20405,1
33206,201701,20566,1
48057,201701,20073,1
57610,201701,20012,1
58353,201701,20235,1
...,...,...,...
2931322,201912,20491,1
2934459,201912,20681,1
2936524,201912,20286,1
2944522,201912,20567,1


In [75]:
# add a column "plan_precios_cuidados" to df_desagregado_final based on the values in df_precios_cuidados
df_desagregado_final = pd.merge(df_desagregado_final, df_precios_cuidados, on=['periodo', 'product_id'], how='left', validate="many_to_many")

In [76]:
df_desagregado_final.shape

(19109373, 12)

In [77]:

df_desagregado_final.fillna(0, inplace=True)

In [78]:
# convert plan_precios_cuidados to int
df_desagregado_final['plan_precios_cuidados'] = df_desagregado_final['plan_precios_cuidados'].astype(int)
df_desagregado_final["plan_precios_cuidados"].value_counts()

plan_precios_cuidados
0    18925497
1      183876
Name: count, dtype: int64

In [79]:
# show rows with "unknown" in any column
df_desagregado_final[df_desagregado_final.isin(['unknown']).any(axis=1)]

,periodo,customer_id,product_id,cust_request_qty,cust_request_tn,tn,product_category,cat2,cat3,brand,sku_size,plan_precios_cuidados
177,201701,10001,20808,4.0,0.54939,0.54939,unknown,unknown,unknown,unknown,0.0,0
201,201701,10001,20848,4.0,0.25946,0.25946,unknown,unknown,unknown,unknown,0.0,0
306,201701,10001,21066,0.0,0.00000,0.00000,unknown,unknown,unknown,unknown,0.0,0
324,201701,10001,21098,3.0,0.26743,0.26743,unknown,unknown,unknown,unknown,0.0,0
330,201701,10001,21125,6.0,0.04050,0.04050,unknown,unknown,unknown,unknown,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
19107940,201912,10572,21169,0.0,0.00000,0.00000,unknown,unknown,unknown,unknown,0.0,0
19107973,201912,10572,21228,0.0,0.00000,0.00000,unknown,unknown,unknown,unknown,0.0,0
19108744,201912,10582,20918,0.0,0.00000,0.00000,unknown,unknown,unknown,unknown,0.0,0
19108899,201912,10582,21169,0.0,0.00000,0.00000,unknown,unknown,unknown,unknown,0.0,0


In [80]:
# show rows with "unknown" in any column
df_desagregado_final[df_desagregado_final.isin(['unknown']).any(axis=1)]["product_id"].unique()

array([20808, 20848, 21066, 21098, 21125, 21165, 21178, 21199, 21217,
       21223, 21225, 21230, 21238, 21240, 21241, 21249, 21253, 21268,
       21272, 21273, 21274, 21275, 21277, 21285, 21286, 21295, 21297,
       21281, 21278, 21284, 21279, 21283, 21289, 21290, 21288, 21291,
       21292, 21293, 21296, 21298, 21299, 20918, 21228, 21270, 21169])

In [81]:
df_desagregado_final["brand"]

0            SHAMPOO1
1           CAPILATIS
2            SHAMPOO3
3               DEOS3
4             MUSCULO
              ...    
19109368     SHAMPOO1
19109369     SHAMPOO2
19109370      MUSCULO
19109371      MUSCULO
19109372      MUSCULO
Name: brand, Length: 19109373, dtype: object

In [82]:
df_desagregado_final.shape

(19109373, 12)

In [83]:
df_desagregado_final.to_csv("./datasets/tb_sellout_02_desglosado.csv", index=False)